In [1]:
# Library

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
train = pd.read_csv(r"E:\INCEPTEZ\Training\Basics_python\Big_mart_linear_regression\train_v9rqX0R.csv")

In [3]:
test = pd.read_csv(r"E:\INCEPTEZ\Training\Basics_python\Big_mart_linear_regression\test_AbJTz2l.csv")

In [4]:
sample_submission= pd.read_csv(r"E:\INCEPTEZ\Training\Basics_python\Big_mart_linear_regression\sample_submission_8RXa3c6.csv")

In [5]:
train.head()



,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [6]:
train.shape

(8523, 12)

In [7]:
test.shape

(5681, 11)

In [8]:
sample_submission.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1000
1,FDW14,OUT017,1000
2,NCN55,OUT010,1000
3,FDQ58,OUT017,1000
4,FDY38,OUT027,1000


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [10]:
train.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Item_Identifier,8523,1559,FDW13,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Weight,7060.0,NaN,NaN,NaN,12.857645,4.643456,4.555,8.77375,12.6,16.85,21.35
Item_Fat_Content,8523,5,Low Fat,5089,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_Visibility,8523.0,NaN,NaN,NaN,0.066132,0.051598,0.0,0.026989,0.053931,0.094585,0.328391
Item_Type,8523,16,Fruits and Vegetables,1232,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Item_MRP,8523.0,NaN,NaN,NaN,140.992782,62.275067,31.29,93.8265,143.0128,185.6437,266.8884
Outlet_Identifier,8523,10,OUT027,935,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Establishment_Year,8523.0,NaN,NaN,NaN,1997.831867,8.37176,1985.0,1987.0,1999.0,2004.0,2009.0
Outlet_Size,6113,3,Medium,2793,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Outlet_Location_Type,8523,3,Tier 3,3350,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<!-- Problem:
    1. Item_Identifier - Object, lot of unique
    2. Item_Weight - Null value, 
    3. Item_Fat_Content - Object, 
    4. Item_Visibility - no visibility product aswell
    5. Item_Type - 
    6. Item_MRP - 
    7. Outlet_Identifier - Object
    8. Year - covert to age
    9. Size - NA, objevt 
    10. location_type - Object
    11. outlet_type - object -->

In [11]:
# Assumptions or Intution :
#     Outlet : 
#         Location - Yes
#         Population - No
#         Type - Yes
#         Size - Yes
#         Age - Yes
#         Discounts - Yes(MRP)
        
#     Item :
#         Visibility - yes
#         Price - yes
#         Size - Yes
#         Brand - No
#         Age - NO
#         Type- Yes
#         Fat_content - Yes

In [12]:
# Train
# A B C
# f 1 34
# g 3 64
# f 4 32
# g 6 12
# k 6 54


# OneHot encode separately - ?

# A_f A_g A_k B C
# 1    0   0  1. 34
# 0    1   0
# 1    0   0
# 0    1   0
# 0    0   1

In [13]:
# Test
# A B 
# f 1
# g 5
# i 7

# OneCode encode separately

# A_f. A_g A_i  B
# 1     0   0
# 0     1   0
# 0     0   1

In [14]:
# A B C
# f 1 34
# g 3 64
# f 4 32
# g 6 12
# k 6 54
# f 1
# g 5
# i 7

# encode?
# A_f A_g A_k A_i B C    source
# 1  0.    0.  0. 11 32   train
# 0. 0.    1.  0. 12. 43   train
# ;;;
# ;;;;
# 0  0.    0.  1. 32  ?    test


# train = data[data.source ='train']
# test = data[data.source = 'test']
# drop source column
# Modelling - model
# A_f A_g A_k A_i B C
# 1  0.    0.  0. 11 32
# 0. 0.    1.  0. 12. 43

In [15]:
train['source'] = 'train'
test['source'] = 'test'

In [16]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train


In [17]:
data = pd.concat([train,test])

In [18]:
data.shape

(14204, 13)

In [19]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train


In [20]:
data.isna().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [21]:
# item.       weight
# soap          20
# cotton         1
# iron box      1000
# soap           30
# cotton          2
# cotton.         3
# iron box       1200
# iron box       ? 3 1180 
# cotton.        ? 3 1180



# soap   20
# soap   30
 
#     25

# cotton  1
# cotton  2
# cotton. 3
  
#     2
    

In [22]:
data.groupby(['Item_Identifier', 'Item_Type']).Item_Weight.mean().to_frame()

,,Item_Weight
Item_Identifier,Item_Type,
DRA12,Soft Drinks,11.600
DRA24,Soft Drinks,19.350
DRA59,Soft Drinks,8.270
DRB01,Soft Drinks,7.390
DRB13,Soft Drinks,6.115
...,...,...
NCZ30,Household,6.590
NCZ41,Health and Hygiene,19.850
NCZ42,Household,10.500


In [23]:
data[data['Item_Weight'].isna()]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636,train
18,DRI11,NaN,Low Fat,0.034238,Hard Drinks,113.2834,OUT027,1985,Medium,Tier 3,Supermarket Type3,2303.6680,train
21,FDW12,NaN,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432,train
23,FDC37,NaN,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876,train
29,FDC14,NaN,Regular,0.072222,Canned,43.6454,OUT019,1985,Small,Tier 1,Grocery Store,125.8362,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5655,FDX32,NaN,Regular,0.099375,Fruits and Vegetables,143.6786,OUT027,1985,Medium,Tier 3,Supermarket Type3,NaN,test
5656,FDC39,NaN,Low Fat,0.158425,Dairy,207.8296,OUT027,1985,Medium,Tier 3,Supermarket Type3,NaN,test
5661,DRG37,NaN,Low Fat,0.000000,Soft Drinks,155.7972,OUT027,1985,Medium,Tier 3,Supermarket Type3,NaN,test
5665,DRE03,NaN,Low Fat,0.024110,Dairy,46.0718,OUT027,1985,Medium,Tier 3,Supermarket Type3,NaN,test


In [24]:
data.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,train
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528,train
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636,train
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986,train
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350,train


In [25]:
data['Item_Weight'] = data.groupby(['Item_Identifier', 'Item_Type']).Item_Weight.transform(lambda x: x.fillna(x.mean()))

In [26]:
data.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,train
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528,train
7,FDP10,19.000,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636,train
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986,train
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350,train


In [27]:
data.isna().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [28]:
data['Item_Fat_Content'].value_counts()

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

In [29]:
data['Item_Fat_Content'].replace({'LF' :'Low Fat', 'low fat':'Low Fat', 'reg':'Regular'}, inplace = True)

In [30]:
data['Item_Fat_Content'].value_counts()

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

In [31]:
data['Outlet_Establishment_Year'] 

0       1999
1       2009
2       1999
3       1998
4       1987
        ... 
5676    1997
5677    2009
5678    2002
5679    2007
5680    2002
Name: Outlet_Establishment_Year, Length: 14204, dtype: int64

In [32]:
data['Outlet_Age'] = 2022 - data['Outlet_Establishment_Year']

In [33]:
data['Outlet_Age']

0       23
1       13
2       23
3       24
4       35
        ..
5676    25
5677    13
5678    20
5679    15
5680    20
Name: Outlet_Age, Length: 14204, dtype: int64

In [34]:
data['Outlet_Size'].value_counts()

Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

In [35]:
data.groupby(['Outlet_Type']).Outlet_Size.agg(pd.Series.mode)

Outlet_Type
Grocery Store         Small
Supermarket Type1     Small
Supermarket Type2    Medium
Supermarket Type3    Medium
Name: Outlet_Size, dtype: object

In [36]:
# 1 2 3 4 1 2

# 1 2

In [37]:
data['Outlet_Size'] = data.groupby(['Outlet_Type']).Outlet_Size.transform(lambda x: x.fillna(x.mode()[0]))

In [38]:
data.isna().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
Outlet_Age                      0
dtype: int64

In [39]:
data['Outlet_Location_Type'].value_counts()

Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

In [40]:
data['Item_Identifier'].value_counts()

FDU15    10
FDS25    10
FDA38    10
FDW03    10
FDJ10    10
         ..
FDR51     7
FDM52     7
DRN11     7
FDH58     7
NCW54     7
Name: Item_Identifier, Length: 1559, dtype: int64

In [41]:
data['Item_Identifier'].str[0:2].value_counts()

FD    10201
NC     2686
DR     1317
Name: Item_Identifier, dtype: int64

In [42]:
data['Item_Identifer_code'] = data['Item_Identifier'].str[0:2]

In [48]:
data_numbers = data.select_dtypes(include=np.number).drop(columns='Outlet_Establishment_Year')

In [49]:
scale = StandardScaler()
scaled = scale.fit_transform(data_numbers.drop(columns='Item_Outlet_Sales'))
data_numbers = pd.DataFrame(scaled, columns=data_numbers.drop(columns='Item_Outlet_Sales').columns)

Ordinal  - Order     ex: Rating, Rank   -   Label Encoding - 1 ,2 ,3 ,4, 5
Nominal  - No Order  ex: Gender, color  -   One Hot Encoding
 

In [50]:
data_category = data.select_dtypes(exclude=np.number)

In [51]:
data_category_one_hot = pd.get_dummies(data_category.drop(columns=['source','Item_Identifier']))

In [52]:
data_preprocessed = pd.concat([data_numbers, data_category_one_hot, data[['Item_Identifier','Outlet_Identifier','source','Item_Outlet_Sales']]], axis = 1)

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
train_preprocessed = data_preprocessed[data_preprocessed['source']=='train'].drop(columns='source')
test_preprocessed = data_preprocessed[data_preprocessed['source']=='test'].drop(columns=['source', 'Item_Outlet_Sales'])


In [ ]:
train_preprocessed

In [ ]:
test_preprocessed

In [ ]:
train_preprocessed.to_csv("train_preprocessed.csv", index = False)
test_preprocessed.to_csv("test_preprocessed.csv", index = False)

### Modelling

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [ ]:
train = pd.read_csv(r'E:\INCEPTEZ\Training\Basics_python\Big_mart_linear_regression\train_preprocessed.csv')

In [ ]:
test= pd.read_csv(r'E:\INCEPTEZ\Training\Basics_python\Big_mart_linear_regression\test_preprocessed.csv')

In [ ]:
train.head()

In [ ]:
x =train.drop(columns=['Item_Outlet_Sales','Item_Identifier','Outlet_Identifier'])

In [ ]:
y = train['Item_Outlet_Sales']

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2, random_state=1)

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(x_train,y_train)

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
x_train_pred= model.predict(x_train)
x_test_pred= model.predict(x_test)

## Actual test data from Hackathon
test_pred = model.predict(test.drop(columns=['Item_Identifier','Outlet_Identifier']))

In [ ]:
train_rmse = np.sqrt(mean_squared_error(y_train,x_train_pred))
train_rmse

In [ ]:
test_rmse = np.sqrt(mean_squared_error(y_test,x_test_pred))
test_rmse

In [ ]:
r2_score(y_train,x_train_pred)

In [ ]:
r2_score(y_test,x_test_pred)

In [ ]:
test['Item_Outlet_Sales'] = test_pred

In [ ]:
output = test[['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']]

In [ ]:
output.to_csv('Result_linear_Regression_with_80percent_testdata')